In [272]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from sklearn.model_selection import train_test_split

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout

# 데이터 로드

In [273]:
curr_dir = os.getcwd()

forders_tr = os.listdir(curr_dir+"\\data")

data = pd.read_csv(os.path.join(curr_dir+"\\data\\")+forders_tr[0])
data = data.drop(data.columns[0], axis=1)
data

submission = pd.read_csv('2-1_검증데이터셋.csv')
submission1 = submission.copy()          

# 전처리

In [121]:
soil = pd.read_csv('OBS_AAOS_TIM_20220802210224.csv', encoding='cp949')       # 외부 데이터 로드
stn = list(set(soil['지점'].values))

soil_df1 = []      # 일시 포함된 데이터 프레임
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    soil_df1.append(df)

    
soil_df = []        # 일시가 제외된 데이터 프레임 (추합을 위해)
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df = df.drop(['지점', '지점명', '일시'], axis=1)   
    soil_df.append(df) 
  
day = pd.date_range('2021-11-26', periods=95*24+20*24, freq='1H')    # 21.11.26~22.03.20
day
day1 = pd.DataFrame(day)
day1.columns = ['일자']
day1
day1.to_csv('day_real.csv', index=False)


for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df.to_csv(os.path.join(str(stn[i])+'.csv'), index=False)   # 관측소마다.csv 파일로 저장

In [251]:
# 관측소마다 일시(일자) 통합을 위해 'day_real'파일과 위에서 저장된 모든 관측소에 대해 엑셀 VLOOKUP 함수 이용하여 
# 중간에 비어 있는 일시를 채워주는 작업 후, 각 파일을 _1.csv 파일로 저장

soil_df2 = []
for i in range(len(stn)):
    df = pd.read_csv(os.path.join(str(stn[i])+'_1.csv'), encoding='cp949')
    df = df.drop('일자', axis=1)
    df = df.fillna(0)
    soil_df2.append(df)



soil_df3 = soil_df2[0].copy()
for i in range(len(soil_df[0].columns)):
    soil_df2[0][soil_df2[0].columns[i]] = 0

# 각 관측소마다 평균 내기 (추합)
for i in range(1, len(soil_df2)):
    soil_df3 = soil_df3 + soil_df2[i]
soil_df3 = soil_df3/len(soil_df2)
soil_df3['일자'] = pd.date_range('2021-11-26', periods=95*24+20*24, freq='1H')


# 제공 데이터와의 통합을 위해 제공 데이터에 없는 일시를 외부 데이터에서 제외시킴
d1 = soil_df3[soil_df3['일자'] == '2021-12-22 0:00'].index[0]
for i in range(d1, d1+24*3):
    soil_df3 = soil_df3.drop(i,axis=0)

d2 = soil_df3[soil_df3['일자'] == '2022-03-01 0:00'].index[0]
for i in range(d2, d2+24*3):
    soil_df3 = soil_df3.drop(i,axis=0)
    
d3 = soil_df3[soil_df3['일자'] == '2022-03-14 0:00'].index[0]
for i in range(d3, d3+24):
    soil_df3 = soil_df3.drop(i,axis=0)
    
d4 = soil_df3[soil_df3['일자'] == '2022-03-16 0:00'].index[0]
for i in range(d4, d4+24*2):
    soil_df3 = soil_df3.drop(i,axis=0)

soil_df3 = soil_df3.reset_index()
soil_df3 = soil_df3.drop('index', axis=1)

# 제공 데이터와 외부 데이터 통합 (시간 데이터 제외 모든 열)
col = ['smart_farm.insolation', 'smart_farm.out_tmperature',\
       'smart_farm.out_humidity', 'smart_farm.wind_speed',\
       'smart_farm.wind_direction', 'smart_farm.in_tmperature',\
       'smart_farm.in_humidity', 'smart_farm.shield_light_h',\
       'smart_farm.shield_tmperature_h', 'smart_farm.shield_energy_h',\
       'smart_farm.shield_energy_v', 'smart_farm.exhaust_fan',\
       'smart_farm.ceiling', 'smart_farm.floating_fan',\
       'smart_farm.fan_coil_b_site',\
       'smart_farm.ventilation_temperature_control',\
       'smart_farm.heating_temperature_set_up']          

data_ = pd.DataFrame()
for k in tqdm(range(len(col))):
    h = []
    for i in range(int(len(data)/60)):      # 분 -> 시간 단위 바꾸기 위해
        h.append(np.mean(data[col[k]][60*i:60*(i+1)]))
    data_[col[k]] = h
    
h1 = []
for i in range(int(len(data)/60)):
    h1.append(np.sum(data['smart_farm.heat_supply'][60*i:60*(i+1)]))
data_['smart_farm.heat_supply'] = h1



real_data = pd.concat([soil_df3, data_], axis=1)
real_data1 = pd.concat([real_data[real_data.columns[4]], real_data[real_data.columns[:4]], real_data[real_data.columns[5:]]], axis=1)
real_data1 = real_data1.reset_index()
real_data1 = real_data1.drop('index', axis=1)
real_data1 = real_data1.dropna(subset = ['일자'])
real_data1.to_csv('real_data_통합.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  2.86it/s]


In [274]:
# 위에서 저장한 'real_data_통합' 파일과 'day_real' 파일 이용하여
# 엑셀 VLOOKUP 함수 이용하여 21.11.26부터 22.03.20까지 비어 일자 채운 후, 다시 'real_data_결측' 파일로 저장

pre = pd.read_csv('real_data_결측.csv', encoding='cp949')
pre1 = pre.interpolate(method ='linear')      
pre1 = pre1.dropna(subset = ['일자'])
pre1.to_csv('real_data_최종.csv', index=False)  # 최종 전처리 파일(결측치 제거한)

real1 = pd.read_csv('real_data_최종.csv', encoding='utf-8')
real1

# 자른 이유? 20개를 추출하기 위해 일자 슬라이싱이 필요함.
training = real1[:-504]
testing = real1[-504:]
testing = testing.reset_index()
testing = testing.drop('index', axis=1)

# 모델 1

In [275]:
X = training.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)  

validation = testing.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)


x_train = []
y_train = []

for i in range(24, training_data.shape[0]):
  x_train.append(training_data[i-24:i])
  y_train.append(training_data.iloc[i,-1])
    
    
x_test = []
y_test = []

for i in range(24, test_data.shape[0]):
  x_test.append(test_data[i-24:i])
  y_test.append(test_data.iloc[i,-1])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)


x_validation = []
for i in range(24, validation.shape[0]):
  x_validation.append(validation[i-24:i])
x_validation = np.array(x_validation)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_validation.shape)

regression_GRU = Sequential()
regression_GRU.add(GRU(units=50, activation="relu", return_sequences=True, input_shape = (x_train.shape[1], x_train.shape[2]))) 
regression_GRU.add(Dropout(0.2))

regression_GRU.add(GRU(units=60, activation="relu", return_sequences=True))
regression_GRU.add(Dropout(0.3))

regression_GRU.add(GRU(units=80, activation="relu", return_sequences=True))
regression_GRU.add(Dropout(0.4))

regression_GRU.add(GRU(units=48, activation="relu"))  
regression_GRU.add(Dropout(0.5))

regression_GRU.add(Dense(units = 1))     

regression_GRU.compile(optimizer='adam', loss='mean_squared_error')
regression_GRU.summary()

(1555, 24, 17)
(1555,)
(653, 24, 17)
(653,)
(480, 24, 17)
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_16 (GRU)                (None, 24, 50)            10350     
                                                                 
 dropout_16 (Dropout)        (None, 24, 50)            0         
                                                                 
 gru_17 (GRU)                (None, 24, 60)            20160     
                                                                 
 dropout_17 (Dropout)        (None, 24, 60)            0         
                                                                 
 gru_18 (GRU)                (None, 24, 80)            34080     
                                                                 
 dropout_18 (Dropout)        (None, 24, 80)            0         
                                                              

In [264]:
X = training.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)  

validation = testing.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)


x_train = []
y_train = []

for i in range(24, training_data.shape[0]):
  x_train.append(training_data[i-24:i])
  y_train.append(training_data.iloc[i,-1])
    
    
x_test = []
y_test = []

for i in range(24, test_data.shape[0]):
  x_test.append(test_data[i-24:i])
  y_test.append(test_data.iloc[i,-1])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)


x_validation = []
for i in range(24, validation.shape[0]):
  x_validation.append(validation[i-24:i])
x_validation = np.array(x_validation)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_validation.shape)

regression_GRU = Sequential()
regression_GRU.add(GRU(units=50, activation="relu", return_sequences=True, input_shape = (x_train.shape[1], x_train.shape[2]))) 
regression_GRU.add(Dropout(0.2))

regression_GRU.add(GRU(units=60, activation="relu", return_sequences=True))
regression_GRU.add(Dropout(0.3))

regression_GRU.add(GRU(units=80, activation="relu", return_sequences=True))
regression_GRU.add(Dropout(0.4))

regression_GRU.add(GRU(units=48, activation="relu"))  
regression_GRU.add(Dropout(0.5))

regression_GRU.add(Dense(units = 1))     

regression_GRU.compile(optimizer='adam', loss='mean_squared_error')

GRU_history = regression_GRU.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

(1555, 24, 17)
(1555,)
(653, 24, 17)
(653,)
(480, 24, 17)
Epoch 1/10
49/49 [==============================] - 7s 46ms/step - loss: 81.5995 - val_loss: 69.6799
Epoch 2/10
49/49 [==============================] - 2s 42ms/step - loss: 30.3600 - val_loss: 41.0787
Epoch 3/10
49/49 [==============================] - 2s 36ms/step - loss: 24.5216 - val_loss: 64.2694
Epoch 4/10
49/49 [==============================] - 2s 40ms/step - loss: 20.4403 - val_loss: 44.1206
Epoch 5/10
49/49 [==============================] - 2s 41ms/step - loss: 18.7952 - val_loss: 42.5584
Epoch 6/10
49/49 [==============================] - 2s 35ms/step - loss: 17.7417 - val_loss: 34.4056
Epoch 7/10
49/49 [==============================] - 2s 41ms/step - loss: 17.2556 - val_loss: 19.5026
Epoch 8/10
49/49 [==============================] - 2s 42ms/step - loss: 17.7748 - val_loss: 24.9067
Epoch 9/10
49/49 [==============================] - 2s 37ms/step - loss: 17.4735 - val_loss: 20.8666
Epoch 10/10
49/49 [==============

In [265]:
p = regression_GRU.predict(x_validation)  
p = p.flatten()

h1 = []
for i in range(int(len(p)/24)): 
    h1.append(np.sum(p[24*i:24*(i+1)]))    
    
h1.insert(0,0)     # 한 번만 클릭!
h11 = h1[4:14] + h1[18:21]

submission1['heat_supply_day1'] = h11
submission1 

15/15 [==============================] - 1s 10ms/step


,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,315.190552,NaN
1,2022,3,5,357.271179,NaN
2,2022,3,6,354.682648,NaN
3,2022,3,7,353.583832,NaN
4,2022,3,8,324.192444,NaN
5,2022,3,9,350.268799,NaN
6,2022,3,10,345.567993,NaN
7,2022,3,11,352.262939,NaN
8,2022,3,12,361.658142,NaN
9,2022,3,13,356.693146,NaN


# 모델 2(절감 모델)

In [266]:
X = training.drop(['일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)   # shuffle은 안 됨. 시간적순서라

validation = testing.drop(['일자','smart_farm.heat_supply'], axis=1)


x_train = []
y_train = []

for i in range(24, training_data.shape[0]):
  x_train.append(training_data[i-24:i])
  y_train.append(training_data.iloc[i,-1])
    
x_test = []
y_test = []

for i in range(24, test_data.shape[0]): 
  x_test.append(test_data[i-24:i])
  y_test.append(test_data.iloc[i,-1])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)



x_validation = []
for i in range(24, validation.shape[0]):
  x_validation.append(validation[i-24:i])
x_validation = np.array(x_validation)


print(x_train.shape) 
print(y_train.shape) 
print(x_test.shape)
print(y_test.shape)
print(x_validation.shape)


regression_GRU = Sequential()
regression_GRU.add(GRU(units=50, activation="relu", return_sequences=True, input_shape = (x_train.shape[1], x_train.shape[2]))) 
regression_GRU.add(Dropout(0.2))

regression_GRU.add(GRU(units=60, activation="relu", return_sequences=True))
regression_GRU.add(Dropout(0.3))

regression_GRU.add(GRU(units=80, activation="relu", return_sequences=True))
regression_GRU.add(Dropout(0.4))

regression_GRU.add(GRU(units=48, activation="relu"))   
regression_GRU.add(Dropout(0.5))

regression_GRU.add(Dense(units = 1))    

regression_GRU.compile(optimizer='adam', loss='mean_squared_error')

GRU_history = regression_GRU.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

(1555, 24, 21)
(1555,)
(653, 24, 21)
(653,)
(480, 24, 21)
Epoch 1/10
49/49 [==============================] - 10s 50ms/step - loss: 71.2451 - val_loss: 128.1718
Epoch 2/10
49/49 [==============================] - 2s 44ms/step - loss: 38.1962 - val_loss: 107.2686
Epoch 3/10
49/49 [==============================] - 2s 41ms/step - loss: 28.9111 - val_loss: 101.9744
Epoch 4/10
49/49 [==============================] - 2s 40ms/step - loss: 22.5355 - val_loss: 81.7912
Epoch 5/10
49/49 [==============================] - 2s 42ms/step - loss: 22.4360 - val_loss: 77.6120
Epoch 6/10
49/49 [==============================] - 2s 39ms/step - loss: 18.0669 - val_loss: 60.3737
Epoch 7/10
49/49 [==============================] - 2s 40ms/step - loss: 16.8302 - val_loss: 55.7059
Epoch 8/10
49/49 [==============================] - 2s 42ms/step - loss: 15.4652 - val_loss: 43.1457
Epoch 9/10
49/49 [==============================] - 2s 38ms/step - loss: 13.9368 - val_loss: 32.9341
Epoch 10/10
49/49 [==========

In [267]:
p2 = regression_GRU.predict(x_validation)  
p2 = p2.flatten()

h2 = []
for i in range(int(len(p2)/24)): 
    h2.append(np.sum(p2[24*i:24*(i+1)]))
h2.insert(0,0)     # 한 번만 클릭!
h22 = h2[4:14] + h2[18:21]
h22
submission1['heat_supply_day2'] = h22
submission1   

15/15 [==============================] - 1s 11ms/step


,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,315.190552,311.913940
1,2022,3,5,357.271179,306.018372
2,2022,3,6,354.682648,308.111755
3,2022,3,7,353.583832,303.481689
4,2022,3,8,324.192444,322.216125
5,2022,3,9,350.268799,325.490234
6,2022,3,10,345.567993,320.821655
7,2022,3,11,352.262939,290.205536
8,2022,3,12,361.658142,331.821442
9,2022,3,13,356.693146,305.158447


In [269]:
submission1.to_csv('220224_최종코드틀_GRU.csv', index=False) 